In [1]:
import pandas as pd
import numpy as np

# -----------------------------
# Generate devices.csv
# -----------------------------
device_ids = [f"D{str(i).zfill(3)}" for i in range(1, 51)]  # 50 devices
homes = [f"H{str(i).zfill(3)}" for i in range(1, 21)]       # 20 homes
device_types = ["SmartPlug", "SmartAC", "SmartHeater", "SmartWasher", "SmartFridge", "SolarPanel"]
rooms = ["LivingRoom", "Bedroom", "Kitchen", "Laundry", "Garage"]

devices_data = {
    "device_id": device_ids,
    "home_id": np.random.choice(homes, len(device_ids)),
    "device_type": np.random.choice(device_types, len(device_ids)),
    "room": np.random.choice(rooms, len(device_ids))
}

devices_df = pd.DataFrame(devices_data)
devices_df.to_csv("devices.csv", index=False)

print("devices.csv created with", len(devices_df), "rows")

# -----------------------------
# Generate energy_readings.csv
# -----------------------------
n = 10000  # number of readings
data = {
    "reading_id": [f"R{str(i).zfill(5)}" for i in range(1, n+1)],
    "device_id": np.random.choice(device_ids, n),
    "timestamp": pd.date_range("2025-01-01", periods=n, freq="H"),
    "energy_kwh": np.random.uniform(0.1, 5.0, n).round(2),
    "region": np.random.choice(["North", "South", "East", "West"], n),
    "customer_segment": np.random.choice(["Residential", "Commercial"], n)
}

energy_df = pd.DataFrame(data)
energy_df["cost_in_inr"] = (energy_df["energy_kwh"] * 8).round(2)  # assume ₹8 per kWh

energy_df.to_csv("energy_readings.csv", index=False)

print("energy_readings.csv created with", len(energy_df), "rows")

devices.csv created with 50 rows


/tmp/ipython-input-433887511.py:31: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  "timestamp": pd.date_range("2025-01-01", periods=n, freq="H"),


energy_readings.csv created with 10000 rows


In [2]:
from google.colab import files
files.download('devices.csv')
files.download('energy_readings.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [3]:
devices_df.head()

,device_id,home_id,device_type,room
0,D001,H010,SmartPlug,Kitchen
1,D002,H005,SmartPlug,Kitchen
2,D003,H005,SmartAC,Laundry
3,D004,H011,SmartWasher,Garage
4,D005,H009,SolarPanel,Laundry


In [4]:
energy_df.head()

,reading_id,device_id,timestamp,energy_kwh,region,customer_segment,cost_in_inr
0,R00001,D038,2025-01-01 00:00:00,1.46,South,Residential,11.68
1,R00002,D039,2025-01-01 01:00:00,1.81,North,Residential,14.48
2,R00003,D041,2025-01-01 02:00:00,1.21,East,Residential,9.68
3,R00004,D004,2025-01-01 03:00:00,3.06,East,Commercial,24.48
4,R00005,D002,2025-01-01 04:00:00,2.46,North,Commercial,19.68


In [5]:
import pandas as pd
import sqlite3

# Load CSVs into Pandas
devices_df = pd.read_csv("devices.csv")
energy_df = pd.read_csv("energy_readings.csv")

print("Devices sample:")
print(devices_df.head())

print("\nEnergy readings sample:")
print(energy_df.head())

# Create SQLite database
conn = sqlite3.connect("energy_project.db")

# Write dataframes to SQLite tables
devices_df.to_sql("devices", conn, if_exists="replace", index=False)
energy_df.to_sql("energy_readings", conn, if_exists="replace", index=False)

print("\nTables created in SQLite: devices, energy_readings")

# Run a sample SQL query
query = """
SELECT region, customer_segment, COUNT(*) as total_readings,
       AVG(energy_kwh) as avg_usage
FROM energy_readings
GROUP BY region, customer_segment
ORDER BY avg_usage DESC;
"""

result = pd.read_sql_query(query, conn)
print("\nSample aggregated results:")
print(result)

Devices sample:
  device_id home_id  device_type     room
0      D001    H010    SmartPlug  Kitchen
1      D002    H005    SmartPlug  Kitchen
2      D003    H005      SmartAC  Laundry
3      D004    H011  SmartWasher   Garage
4      D005    H009   SolarPanel  Laundry

Energy readings sample:
  reading_id device_id            timestamp  energy_kwh region  \
0     R00001      D038  2025-01-01 00:00:00        1.46  South   
1     R00002      D039  2025-01-01 01:00:00        1.81  North   
2     R00003      D041  2025-01-01 02:00:00        1.21   East   
3     R00004      D004  2025-01-01 03:00:00        3.06   East   
4     R00005      D002  2025-01-01 04:00:00        2.46  North   

  customer_segment  cost_in_inr  
0      Residential        11.68  
1      Residential        14.48  
2      Residential         9.68  
3       Commercial        24.48  
4       Commercial        19.68  

Tables created in SQLite: devices, energy_readings

Sample aggregated results:
  region customer_segment 

In [6]:
query1 = """
SELECT region, customer_segment,
       COUNT(*) AS total_readings,
       AVG(energy_kwh) AS avg_usage,
       SUM(cost_in_inr) AS total_cost
FROM energy_readings
GROUP BY region, customer_segment
ORDER BY avg_usage DESC;
"""
pd.read_sql_query(query1, conn)

,region,customer_segment,total_readings,avg_usage,total_cost
0,West,Commercial,1239,2.602897,25799.92
1,North,Commercial,1250,2.599152,25991.52
2,South,Commercial,1227,2.596504,25487.28
3,East,Commercial,1185,2.586464,24519.68
4,South,Residential,1255,2.567060,25773.28
5,North,Residential,1263,2.513547,25396.88
6,West,Residential,1268,2.488604,25244.40
7,East,Residential,1313,2.485918,26112.08


In [7]:
query2 = """
SELECT e.device_id, d.device_type, d.room,
       SUM(e.energy_kwh) AS total_usage
FROM energy_readings e
JOIN devices d ON e.device_id = d.device_id
GROUP BY e.device_id, d.device_type, d.room
ORDER BY total_usage DESC
LIMIT 10;
"""
pd.read_sql_query(query2, conn)

,device_id,device_type,room,total_usage
0,D002,SmartPlug,Kitchen,618.31
1,D025,SolarPanel,Laundry,605.64
2,D031,SmartAC,Garage,583.74
3,D034,SmartAC,LivingRoom,556.73
4,D004,SmartWasher,Garage,555.81
5,D017,SmartHeater,LivingRoom,552.04
6,D021,SmartHeater,LivingRoom,550.20
7,D005,SolarPanel,Laundry,546.23
8,D041,SmartHeater,Garage,543.57
9,D027,SmartFridge,Laundry,543.14


In [8]:
query3 = """
SELECT strftime('%H', timestamp) AS hour,
       AVG(energy_kwh) AS avg_usage
FROM energy_readings
GROUP BY hour
ORDER BY avg_usage DESC;
"""
pd.read_sql_query(query3, conn)

,hour,avg_usage
0,15,2.675204
1,21,2.670889
2,17,2.627332
3,22,2.626899
4,14,2.619568
5,04,2.617170
6,09,2.608753
7,10,2.606283
8,18,2.591587
9,16,2.575240


In [9]:
query5 = """
SELECT customer_segment,
       AVG(energy_kwh) AS avg_usage,
       SUM(cost_in_inr) AS total_cost
FROM energy_readings
GROUP BY customer_segment;
"""
pd.read_sql_query(query5, conn)

,customer_segment,avg_usage,total_cost
0,Commercial,2.596368,101798.40
1,Residential,2.513401,102526.64
